### Solar 10.7B Instruct
#### RAG with LlamaIndex - Nvidia CUDA + WSL (Windows Subsystem for Linux) + Word documents + Local LLM

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation using Windows' WSL and an Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

/home/markwsl/miniconda3/envs/LlamaIndexRAGLinux/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/solar-10.7b-instruct-v1.0.Q5_K_M.gguf',
    temperature=0.1,
    max_new_tokens=1024, # Increasing to support longer responses
    context_window=4096, # Context Window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 35}, # 43 was a good amount of layers for the RTX 3090, you may need to decrease yours if you have less VRAM than 24GB
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 24 key-value pairs and 435 tensors from ./Models/solar-10.7b-instruct-v1.0.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_f

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large", cache_folder=None)

#### 6. Prompt Template

Prompt template for SOLAR 10.7B Instruct:

&#35;&#35;&#35; User:<br>
{prompt}<br>

&#35;&#35;&#35; Assistant:

In [5]:
# Produces a prompt specific to the model
def modelspecific_prompt(promptmessage, questionText):
    return f"### User:\n{promptmessage}{questionText}\n\n###Assistant:"

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256 # Number of tokens in each chunk

#### 8. Index documents

In [7]:
index = VectorStoreIndex.from_documents(documents)

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [8]:
from llama_index.core.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [9]:
def RunQuestion(questionText):
    systemmessage = "You are a story teller who likes to elaborate. Answer questions in a positive, helpful and interesting way. If the answer is not in the following context return ONLY 'Sorry, I don't know the answer to that'. Question: "

    queryQuestion = modelspecific_prompt(systemmessage, questionText)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [10]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [11]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me



llama_print_timings:        load time =    3898.61 ms
llama_print_timings:      sample time =     101.09 ms /   343 runs   (    0.29 ms per token,  3393.05 tokens per second)
llama_print_timings: prompt eval time =    5787.67 ms /  1191 tokens (    4.86 ms per token,   205.78 tokens per second)
llama_print_timings:        eval time =   37533.99 ms /   342 runs   (  109.75 ms per token,     9.11 tokens per second)
llama_print_timings:       total time =   44251.36 ms /  1533 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 1.docx| Source 1:
Thundertooth faced a dilemma – he was hungry, but he couldn't bring himself to feast on the humans who scurried around like ants. As his hunger grew, he stumbled upon a park, an oasis of greenery amidst the concrete and steel. The park was adorned with holographic flowers that emitted a gentle glow, creating an ethereal atmosphere.



While Thundertooth marveled at the beauty of the park, the mayor of the city happened to be passing by. Mayor Eleanor Grace, a charismatic and forward-thinking leader, was immediately intrigued by the sight of the talking dinosaur. She approached Thundertooth with a mix of curiosity and caution.



"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace l


llama_print_timings:        load time =    3898.61 ms
llama_print_timings:      sample time =      44.26 ms /   149 runs   (    0.30 ms per token,  3366.55 tokens per second)
llama_print_timings: prompt eval time =    3082.38 ms /  1070 tokens (    2.88 ms per token,   347.13 tokens per second)
llama_print_timings:        eval time =   15746.58 ms /   148 runs   (  106.40 ms per token,     9.40 tokens per second)
llama_print_timings:       total time =   19216.33 ms /  1218 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated t


llama_print_timings:        load time =    3898.61 ms
llama_print_timings:      sample time =      33.35 ms /   112 runs   (    0.30 ms per token,  3357.92 tokens per second)
llama_print_timings: prompt eval time =    1959.82 ms /   885 tokens (    2.21 ms per token,   451.57 tokens per second)
llama_print_timings:        eval time =   11779.25 ms /   111 runs   (  106.12 ms per token,     9.42 tokens per second)
llama_print_timings:       total time =   14020.71 ms /   996 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 2.docx| Source 1:
Thundertooth



Embraced by the futuristic city and its inhabitants, Thundertooth found a sense of purpose beyond merely satisfying his hunger. Inspired by the advanced technology surrounding him, he decided to channel his creativity into something extraordinary. With the help of the city's brilliant engineers, Thundertooth founded a one-of-a-kind toy factory that produced amazing widgets – magical, interactive toys that captivated the hearts of both children and adults alike.



Thundertooth's toy factory became a sensation, and its creations were highly sought after. The widgets incorporated cutting-edge holographic displays, levitation technology, and even the ability to change shapes and colors with a mere thought. Children across the city rejoiced as they played with these incredible toys that seemed to bring their wildest fantasies to life.



As the years passed, Thundertooth's life took a heartwarming turn. He met a kind and intelligent


llama_print_timings:        load time =    3898.61 ms
llama_print_timings:      sample time =      91.72 ms /   310 runs   (    0.30 ms per token,  3380.00 tokens per second)
llama_print_timings: prompt eval time =    2095.75 ms /   961 tokens (    2.18 ms per token,   458.55 tokens per second)
llama_print_timings:        eval time =   33070.89 ms /   309 runs   (  107.03 ms per token,     9.34 tokens per second)
llama_print_timings:       total time =   35991.08 ms /  1270 tokens


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated t

#### 13. Output responses

In [12]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

In this captivating tale, we encounter Thundertooth, a time-traveling dinosaur who finds himself in the future, grappling with the dilemma of his predatory nature towards humans. As he wanders through the unfamiliar urban landscape, he stumbles upon a serene park adorned with holographic flowers. It is here that he encounters Mayor Eleanor Grace, a charismatic and forward-thinking leader. Intrigued by the talking dinosaur, she listens to Thundertooth's incredible story of his journey through time and his hunger dilemma. Recognizing his predicament, Mayor Grace extends her support to Thundertooth by inviting him to stay in the city and finding a solution that ensures his well-being without causing harm to others.

In another part of the story, we meet Thundertooth alongside his companions Lumina, Echo, Sapphire, and Ignis, who collectively face another challenge - an incoming meteor threatening the city's safety. Each member of the group utilizes their un